Credit: https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/

## Install SpaCy, which is an open-source for NLP in Python

In [20]:
conda install -c conda-forge spacy

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/valerie/opt/anaconda3

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.10.1               |   py37hf985489_0         3.0 MB  conda-forge
    cymem-2.0.2                |   py37h6de7cb9_0          31 KB  conda-forge
    cython-blis-0.2.4          |   py37h01d97ff_1         2.2 MB  conda-forge
    murmurhash-1.0.0         

In [25]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [37]:
# read data
train = pd.read_csv('/Users/valerie/Downloads/train_2kmZucJ.csv')
test = pd.read_csv("/Users/valerie/Downloads/test_oJQbWVk.csv")

train.shape, test.shape

((7920, 3), (1953, 2))

In [38]:
train['label'].value_counts(normalize = True)

0    0.744192
1    0.255808
Name: label, dtype: float64

0: A non-negative tweet, 
1: A negative tweet

In [41]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


If you take a look at id 5, the label is 1. This means this train model recognizes the tweet as a negative. The model seems to work fine especially when you see "for their stupid support!"

In [43]:
# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [50]:
!python -m spacy download en

     |████████████████████████████████| 11.1 MB 8.4 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074433 sha256=5748e1d1e305c18a96e5a275ffe9614b93706464e62626443704c08c7c572e49
  Stored in directory: /private/var/folders/br/z14fx45s7x31tlc24wq2_c0h0000gn/T/pip-ephem-wheel-cache-sfscysvk/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/valerie/opt/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/Users/valerie/opt/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [51]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [52]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])

In [53]:
train.sample(10)

,id,label,tweet,clean_tweet
6212,6213,0,BIRTHDAY BOYYYY him #cute #smile #iphone #nova #15cs birthday #cutest #exi @ Barquisimeto http://instagr.am/p/S0uODnLOyy/,BIRTHDAY boyyyy -PRON- # cute # smile # iphone # nova # 15cs birthday # cutest # exi @ Barquisimeto
2927,2928,0,so i guess #tmobilestillsucks will be a #tt soon..#fail .... i my #iphone now more than ever!!!,so i guess # tmobilestillsuck will be a # tt soon .. #fail .... i -PRON- # iphone now more than ever ! ! !
6825,6826,1,You suck ! #suck #why #why #why #over #thinking #apple #iphone #now #weed http://instagram.com/p/e_tNIROIGf/,-PRON- suck ! # suck # why # why # why # over # think # apple # iphone # now # weed
5850,5851,0,Looks good . #impala #money #clarksvilletn #popcorn #poptart #nike #clarksvilletn #iphone… https://www.instagram.com/p/Bb1-NyIHL5F/,look good . # impala # money # clarksvilletn # popcorn # poptart # nike # clarksvilletn # iphone …
2031,2032,0,#100happydays day12: DIY cord protector. Thanks @airente #cute #samsung #headset #InstaMagAndroid http://instagram.com/p/st_zv7TUUr/,# 100happydays day12 : DIY cord protector . thank @airente # cute # samsung # headset # InstaMagAndroid
4532,4533,0,taken by #XperiaC4 #selfie #moment #U & #I #xperia #myxperia #xperiaid #xperiaz1 #sony… https://instagram.com/p/8vC-y3Sp9I/,take by # XperiaC4 # selfie # moment # U & # -PRON- # xperia # myxperia # xperiaid # xperiaz1 # sony …
4624,4625,0,$&@*# I'm n got a #iPHONE for xMas.... I still got my bbm though... 786# is dead in a few hours holla@me on my 646# don't eva erace dat!,$ & @ * # -PRON- be n get a # iPHONE for xMas .... -PRON- still get -PRON- bbm though ... 786 # be dead in a few hour holla@me on -PRON- 646 # do not eva erace dat !
1084,1085,1,I am getting very frustrated with my iPhone 4... It's time to take it into the store :/ #Apple #Problems,-PRON- be get very frustrated with -PRON- iPhone 4 ... -PRON- be time to take -PRON- into the store :/ # Apple # problem
2535,2536,0,I seriously didn't think itunes could get less visually appealing. #apple,-PRON- seriously do not think itune could get less visually appealing . # apple
4096,4097,1,when my phone dies on me at 50% and it's in the time of an emergency,when -PRON- phone die on -PRON- at 50 % and -PRON- be in the time of an emergency


In [100]:
!pip install 'tensorflow==1.14.0'

     |████████████████████████████████| 105.8 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 488 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 784 kB 4.4 MB/s eta 0:00:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0rc0
    Uninstalling tensorflow-estimator-2.5.0rc0:
      Successfully uninstalled tensorflow-estimator-2.5.0rc0


In [55]:
!pip install tensorflow-hub

     |████████████████████████████████| 108 kB 1.2 MB/s eta 0:00:01


In [110]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=True, signature="default")

In [92]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

In [112]:
# Extract ELMo features 
embeddings = elmo([x])

ValueError: Setting hub.KerasLayer.trainable = True is unsupported when loading from the TF1 Hub format.